In [49]:
import os, glob, json, re
import pandas as pd

In [50]:
records = []

# Append data from all text files in the data_properties directory
for filepath in glob.glob(os.path.join("", "*.txt")):
    with open(filepath, 'r', encoding='utf-8-sig') as f:
        for line in f:
            records.append(json.loads(line))

df = pd.DataFrame(records)
df.describe(include='all')

,project_name,location,location_map
count,643,643,643
unique,642,15,154
top,Acacia Escalades,Quezon City\n ...,"BGC, Taguig, Metro Manila"
freq,2,112,39


In [51]:
# apply string cleaning and corrections
df['location'] = df['location'].str.replace(r'\s{2,}', ' ', regex=True).str.replace(' , ', ', ').str.strip()

# check if the string in 'location_map' contains the corresponding 'location' string, case-insensitive
df['location_match'] = df.apply(lambda row: str(row['location']).lower() in str(row['location_map']).lower(), axis=1)
df

,project_name,location,location_map,location_match
0,Avida Towers Verte,"Taguig, Metro Manila","BGC, Taguig, Metro Manila",True
1,Two Central,"Makati, Metro Manila","Bel-Air, Makati, Metro Manila",True
2,Avida Towers Prime Taft,"Pasay, Metro Manila","Barangay 36, Pasay, Metro Manila",True
3,Sage Residences,"Mandaluyong, Metro Manila","Mauway, Mandaluyong, Metro Manila",True
4,Avida Towers Turf,"Taguig, Metro Manila","BGC, Taguig, Metro Manila",True
...,...,...,...,...
638,The Trion Towers III,"Taguig, Metro Manila","BGC, Taguig, Metro Manila",True
639,The Radiance Manila Bay,"Pasay, Metro Manila","Barangay 3, Pasay, Metro Manila",True
640,The Magnolia Residences,"Quezon City, Metro Manila","Kaunlaran, Quezon City, Metro Manila",True
641,The Sapphire Bloc – South Tower,"Pasig, Metro Manila","San Antonio, Pasig, Metro Manila",True


In [52]:
# Save the DataFrame to a CSV file
df.to_csv("data-properties-for-correction.csv", index=False, encoding='utf-8-sig')